In [ ]:
import serial
import numpy as np
from time import sleep
import sys
import json

COM = 'COM9'# /dev/ttyACM0 (Linux)
BAUD = 19200
# ser.close()
ser = serial.Serial(COM, BAUD, timeout = .1)

print(ser.name)

In [ ]:
def get_and_print_responses(serial_device, print_response = False):
    response = 'No response'
    ret = ''
    while response != '':
        response  = serial_device.readline()
        try:
            response = response.decode('utf-8') 
            ret += response   
        except Exception as e:
            print(f'Exception: {e} \nResponse: {response}')
            
        if print_response:
            print(response)
    return ret
print(get_and_print_responses(ser))
print('--------------------------------------------------------------')
#ser.write(f"Q0\n".encode('utf8'))   # Switch it back-off    
sleep(0.5)  # Sleep a while to make sure data is ready for processing

print(get_and_print_responses(ser))
print('--------------------------------------------------------------')


def getDataSlice(serial_device,adc_type, start_pos):
                 
    commandstring = f'D{adc_type:2}_{start_pos}\n'
    # print(commandstring)
    serial_device.write(commandstring.encode('utf8'))
    json_data = get_and_print_responses(ser, print_response = False)
    # print(json_data, len(json_data))
    data_slice = json.loads(json_data)
    #print("data_slice: ", data_slice, len(data_slice))
    return data_slice   


adc_np_type_map = {1:np.int16,2:np.uint16,3:np.uint16}

def getData(serial_device, stimulus_parameters, adc_type):
    start_pos = 0
    dtype = adc_np_type_map[adc_type]
    length = stimulus_parameters['length']

    # Allocate memory to store the result
    data = np.zeros((length,), dtype=dtype)
    
    while True:
        data_slice = getDataSlice(serial_device,adc_type, start_pos) 
        
        if data_slice['end']<=data_slice['start']:
            break
            
        data[data_slice['start']:data_slice['end']]=np.array(data_slice['data'],dtype=dtype)
        
        start_pos = data_slice['end']    
        
    return data        

def get_stimulus_parameters(serial_device):
    jsonStimulusParameters = None
    stimulus_parameters = None
    try:
        # Get the metadata
        serial_device.write("D00\n".encode('utf8'))
        jsonStimulusParameters = get_and_print_responses(ser)
        stimulus_parameters = json.loads(jsonStimulusParameters)
    except Exception as e:
        print("jsonStimulusParameters: ", jsonStimulusParameters)
        print("stimulus_parameters: ",stimulus_parameters)
        raise(e)
    
    return stimulus_parameters    
    
def get_stimulus_response(serial_device):
    pass

In [ ]:

stimulus_parameters = get_stimulus_parameters(ser)  
print('------------stimulus_parameters----------------')
print(stimulus_parameters) 
print('-----------------------------------------------')
print(getDataSlice(ser,1, 0))
print('-----------------------------------------------')
print(getDataSlice(ser,2, 100))
print('-----------------------------------------------')
print(getDataSlice(ser,3, 2001))  
sleep(0.1)
print('-----------------------------------------------')
print(get_and_print_responses(ser))
V1 = getData(ser, stimulus_parameters, 1)
V2 = getData(ser, stimulus_parameters, 2)
DAC = getData(ser, stimulus_parameters, 3)

In [ ]:
%pylab
plt.plot(np.array(V1),'g.')
plt.plot(np.array(V2),'r.')
plt.plot(np.array(DAC), 'k.')

In [ ]:
# "A" Change the output amplitude used for the measuremnt
ser.write("A0.2\n".encode('utf8'))
response = get_and_print_responses(ser)
print(response)

In [ ]:

response = get_and_print_responses(ser)
print(response)

In [ ]:
# "B" return hardware parameters out: Boardname, bus frequency and CPU frequency
ser.write("B\n".encode('utf8'))
response = get_and_print_responses(ser)
print(response)

In [ ]:
print(get_and_print_responses(ser))

In [ ]:
# "F<freq>\n" Set the DAC output frequency used during a measurement
ser.write("F25\n".encode('utf8'))
response = get_and_print_responses(ser)
print(response)

In [ ]:
# "G<samplefreq>\n" Acquire data at <samplefreq>
ser.write("G50000\n".encode('utf8'))
ser.write("M\n".encode('utf8'))
response = get_and_print_responses(ser)
print(response)

In [ ]:
jsonstr = '{"stimulus":{"length":2000,"f_stimulus":50000.00,"f_sampling":0.00,"ADC_averaging_number":1}}'
d=loads(jsonstring[:-4])
print(jsonstring, d)

In [ ]:

response = get_and_print_responses(ser)
print(response)

In [ ]:
# "Q<level>\n"  
for Vout in range(5, 4096, 409):
    print(f"Q{Vout}\n".encode('utf8'))
    ser.write(f"Q{Vout}\n".encode('utf8'))
    #print('presleep ',end='')
    #print(get_and_print_responses(ser))
    sleep(1)
    #print('postsleep ',end='')
    print(get_and_print_responses(ser))
    
ser.write(f"Q0\n".encode('utf8'))   # Switch it back-off    
print(get_and_print_responses(ser))

In [ ]:
ser.write(f"Q0\n".encode('utf8'))   # Switch it back-off    
get_and_print_responses(ser)

In [ ]:
hex(4095)

In [ ]:
# "Z\n" Perform OCP measurement
ser.write("Z\n".encode('utf8'))
get_and_print_responses(ser)

In [ ]:
# "R\n" 
ser.write("R2600\n".encode('utf8'))
get_and_print_responses(ser)

In [ ]:

get_and_print_responses(ser)

In [ ]:
Vout =4095
print(f"T{Vout:04}\n".encode('utf8'))
ser.write(f"T{Vout:04}\n".encode('utf8'))
get_and_print_responses(ser)
sleep(5.5)
get_and_print_responses(ser)


In [ ]:
import numpy as np
data = [65535,-7502,65535,-6220,65535,-5881,65535,-5078,65535,-7678,65535,-8529,65535,-7560,65535,-6775,65535,-5948,65535,-7850,65535,-6488,65535,-6705,65535,-6385,65535,-8517,65535,-8399,65535,-8993,65535,-9060,65535,-11072,65535,-12655,65535,-12395,65535,-13158,65535,-13070,65535,-14349,65535,-12751,65535,-10723,65535,-7823,65535,-8414,65535,-10188,65535,-10564,65535,-11288,65535,-10195,65535,-10339,65535,-9831,65535,-8999,65535,-10952,65535,-9448,65535,-9308,65535,-11236,65535,-11651,65535,-11631,65535,-10680,65535,-11523,65535,-10924,65535,-9052,65535,-11963,65535,-11708,65535,-10195,65535,-10540,65535,-9016,65535,-9719,65535,-7294,65535,-8696,65535,-7941,65535,-9635,65535,-10536,65535,-9383,65535,-10792,65535,-8768,65535,-8129,65535,-11039,65535,-12854,65535,-10600,65535,-9859,65535,-9751,65535,-7333,65535,-8054,65535,-11235,65535,-11459,65535,-10792,65535,-8864,65535,-8640,65535,-9479,65535,-9479,65535,-9440,65535,-9447,65535,-9271,65535,-8952,65535,-9570,65535,-10088,65535,-11091,65535,-11539,65535,-10039,65535,-12118,65535,-12826,65535,-12570,65535,-13169,65535,-14839,65535,-13249,65535,-11902,65535,-11343,65535,-11331,65535,-12169,65535,-14087,65535,-14063,65535,-15585,65535,-13711,65535,-11091,65535,-11918,65535,-12206,65535,-11218,65535,-12266,65535,-12062,65535,-13887,65535,-13300,65535,-14993,65535,-15853,65535,-14558,65535,-14741,65535,-11726,65535,-11055,65535,-13559,65535,-13815,65535,-12975,65535,-13483,65535,-12078,65535,-12808,65535,-13059,65535,-14837,65535,-12407,65535,-14069,65535,-14469,65535,-13743,65535,-11206,65535,-10774,65535,-9272,65535,-9504,65535,-10691,65535,-10140,65535,-12010,65535,-12991,65535,-14485,65535,-12611,65535,-12183,65535,-12759,65535,-11890,65535,-11975,65535,-12455,65535,-10901,65535,-10043,65535,-13451,65535,-11943,65535,-12375,65535,-13957,65535,-15149,65535,-17061,65535,-16917,65535,-17301,65535,-17445,65535,-14941,65535,-14517,65535,-12914,65535,-13007,65535,-13323,65535,-13455,65535,-11883,65535,-11915,65535,-12087,65535,-14560,65535,-14501,65535,-14085,65535,-16090,65535,-16585,65535,-16110,65535,-17807,65535,-16318,65535,-15301,65535,-16238,65535,-16350,65535,-15501,65535,-15005,65535,-15085,65535,-16633,65535,-15241,65535,-16046,65535,-17185,65535,-15809,65535,-14828,65535,-12914,65535,-12926,65535,-12147,65535,-15113,65535,-17173,65535,-16841,65535,-15529,65535,-15437,65535,-14704,65535,-11955,65535,-13941,65535,-13925,65535,-15069,65535,-15293,65535,-16697,65535,-13551,65535,-14449,65535,-12882,65535,-12695,65535,-11586,65535,-11042,65535,-14389,65535,-14181,65535,-14373,65535,-15553,65535,-15389,65535,-14924,65535,-13682,65535,-12680,65535,-12503,65535,-12686,65535,-11959,65535,-13663,65535,-14476,65535,-13263,65535,-12680,65535,-12594,65535,-13135,65535,-14174,65535,-14624,65535,-15037,65535,-17327,65535,-17743,65535,-15966,65535,-15497,65535,-15393,65535,-16613,65535,-16793,65535,-16417,65535,-15905,65535,-15329,65535,-16281,65535,-17799,65535,-16729,65535,-13737,65535,-11579,65535,-11166,65535,-13163,65535,-14880,65535,-14384,65535,-14336,65535,-15613,65535,-16105,65535,-17800,65535,-16613,65535,-16873,65535,-15889,65535,-13909,65535,-14097,65535,-15049,65535,-17045,65535,-18167,65535,-17547,65535,-17286,65535,-18296,65535,-18444,65535,-15557,65535,-15485,65535,-13300,65535,-15097,65535,-15814,65535,-14686,65535,-13187,65535,-13391,65535,-13439,65535,-13052,65535,-14929,65535,-15489,65535,-13203,65535,-12094,65535,-14063,65535,-15569,65535,-15009,65535,-14603,65535,-14311,65535,-15185,65535,-14715,65535,-11494,65535,-10108,65535,-12388,65535,-13827,65535,-12766,65535,-11590,65535,-10504,65535,-9495,65535,-8336,65535,-8746,65535,-6969,65535,-9392,65535,-9940,65535,-9203,65535,-9111,65535,-10252,65535,-9704,65535,-8832,65535,-8720,65535,-6698,65535,-6666,65535,-8554,65535,-7964,65535,-10492,65535,-10252,65535,-9651,65535,-9635,65535,-10816,65535,-11071,65535,-13322,65535,-13903,65535,-12086,65535,-12794,65535,-10071,65535,-12346,65535,-11828,65535,-11791,65535,-9591,65535,-8896,65535,-8972,65535,-10744,65535,-9955,65535,-10060,65535,-12894,65535,-12122,65535,-12627,65535,-11539,65535,-9971,65535,-11572,65535,-12330,65535,-12966,65535,-12362,65535,-13137,65535,-14663,65535,-13030,65535,-12834,65535,-11836,65535,-11213,65535,-10604,65535,-9490,65535,-8273,65535,-8496,65535,-7416,65535,-9655,65535,-9004,65535,-10716,65535,-9560,65535,-10748,65535,-9591,65535,-8736,65535,-8047,65535,-9447,65535,-9316,65535,-10748,65535,-9463,65535,-8021,65535,-8872,65535,-7598,65535,-7110,65535,-8780,65535,-7150,65535,-8768,65535,-8238,65535,-8085,65535,-9064,65535,-11535,65535,-10332,65535,-11224,65535,-10067,65535,-11859,65535,-9547,65535,-9591,65535,-8696,65535,-8005,65535,-10015,65535,-10712,65535,-9556,65535,-8566,65535,-9895,65535,-9580,65535,-9831,65535,-9259,65535,-9971,65535,-11336,65535,-12015,65535,-13078,65535,-14137,65535,-15706,65535,-13505,65535,-11112,65535,-12834,65535,-11794,65535,-8835,65535,-8453,65535,-8341,65535,-8245,65535,-7902,65535,-7957,65535,-8609,65535,-9388,65535,-10839,65535,-10064,65535,-11864,65535,-9320,65535,-10575,65535,-10719,65535,-8117,65535,-9345,65535,-8853,65535,-6931,65535,-7344,65535,-7587,65535,-7732,65535,-5398,65535,-7327,65535,-8729,65535,-6066,65535,-9093,65535,-9333,65535,-9936,65535,-8981,65535,-9993,65535,-9920,65535,-7635,65535,-6666,65535,-6210,65535,-6834,65535,-6962,65535,-5587,65535,-6221,65535,-5587,65535,-6338,65535,-7400,65535,-8412,65535,-9199,65535,-9941,65535,-8783,65535,-7045,65535,-7109,65535,-7381,65535,-8140,65535,-8412,65535,-9006,65535,-8504,65535,-7975,65535,-7584,65535,-5980,65535,-4195,65535,-6862,65535,-5741,65535,-4500,65535,-2768,65535,-2310,65535,-2607,65535,-4020,65535,-4523,65535,-4323,65535,-5465,65535,-5767,65535,-4443,65535,-4598,65535,-5645,65535,-5789,65535,-3180,65535,-4533,65535,-2976,65535,-2118,65535,-3363,65535,-4523,65535,-4283,65535,-3404,65535,-3586,65535,-3630,65535,-2669,65535,-1795,65535,33,65535,-1126,65535,-2270,65535,-3283,65535,-2694,65535,-2301,65535,-3299,65535,-4051,65535,-4372,65535,-4283,65535,-3443,65535,-3378,65535,-2941,65535,-2750,65535,-1616,65535,-2796,65535,-449,65535,-3454,65535,-4971,65535,-4479,65535,-4772,65535,-3619,65535,-3854,65535,-1737,65535,-1523,65535,-1669,65535,-942,65535,-859,65535,281,65535,-986,65535,-524,65535,-920,65535,-11,65535,-583,65535,-1855,65535,-408,65535,-1126,65535,-3340,65535,-3150,65535,-3920,65535,-1894,65535,-407,65535,-1253,65535,-1892,65535,-482,65535,-1407,65535,-1877,65535,-1381,65535,-1282,65535,-1997,65535,-652,65535,-1873,65535,-1731,65535,-1354,65535,-3647,65535,-4109,65535,-6665,65535,-4836,65535,-3884,65535,-2134,65535,-3251,65535,-2216,65535,-4837,65535,-4106,65535,-6425,65535,-6977,65535,-4763,65535,-1824,65535,-495,65535,-540,65535,320,65535,-1622,65535,-2169,65535,-2830,65535,-704,65535,-622,65535,-1474,65535,-2264,65535,-1872,65535,-1953,65535,-600,65535,-270,65535,-29,65535,-1766,65535,267,65535,45,65535,586,65535,-289,65535,-800,65535,704,65535,-1533,65535,-743,65535,-415,65535,-1619,65535,-2086,65535,-4298,65535,-4570,65535,-2726,65535,-2410,65535,-2717,65535,-3363,65535,-4299,65535,-4676,65535,-5109,65535,-7266,65535,-5793,65535,-2782,65535,-1706,65535,-3596,65535,-2508,65535,-796,65535,816,65535,-1606,65535,-799,65535,777,65535,-986,65535,-1157,65535,-3264,65535,-3698,65535,-2536,65535,-1658,65535,-1984,65535,-2008,65535,-2414,65535,-2439,65535,-2985,65535,-1729,65535,-1557,65535,-1739,65535,-1334,65535,-2555,65535,-2734,65535,-2458,65535,-4282,65535,-3404,65535,-3507,65535,-2028,65535,-2589,65535,-2430,65535,-4085,65535,-4741,65535,-3483,65535,-3671,65535,-4756,65535,-1929,65535,-2759,65535,-841,65535,-729,65535,-834,65535,-1634,65535,-1326,65535,-3203,65535,-5959,65535,-5982,65535,-5629,65535,-4496,65535,-3095,65535,-6057,65535,-6041,65535,-7345,65535,-8120,65535,-9779,65535,-6358,65535,-5945,65535,-4400,65535,-4772,65535,-3454,65535,-5115,65535,-6269,65535,-4949,65535,-3251,65535,-2800,65535,-2656,65535,-2521,65535,-2662,65535,-2159,65535,-5805,65535,-4734,65535,-3887,65535,-2154,65535,-3914,65535,-2608,65535,-3609,65535,-4238,65535,-4003,65535,-5788,65535,-3059,65535,-3857,65535,-4123,65535,-4880,65535,-3165,65535,-5437,65535,-5264,65535,-5525,65535,-4110,65535,-5288,65535,-6336,65535,-8187,65535,-9676,65535,-8204,65535,-8679,65535,-7262,65535,-6445,65535,-7477,65535,-6423,65535,-6621,65535,-5211,65535,-5885,65535,-3839,65535,-3375,65535,-3583,65535,-3959,65535,-4417,65535,-6302,65535,-5493,65535,-4110,65535,-5149,65535,-5958,65535,-4821,65535,-2624,65535,-2327,65535,-3859,65535,-4469,65535,-4821,65535,-2528,65535,-1963,65535,-2766,65535,-2676,65535,-3778,65535,-1804,65535,-1823,65535,-499,65535,-141,65535,-2670,65535,-4302,65535,-4117,65535,-4978,65535,-3466,65535,-4277,65535,-3691,65535,-3306,65535,-3357,65535,-4558,65535,-3719,65535,-3671,65535,-4752,65535,-2560,65535,-2039,65535,-4305,65535,-4323,65535,-5373,65535,-5024,65535,-4943,65535,-3912,65535,-4334,65535,-4293,65535,-5293,65535,-5923,65535,-4513,65535,-4821,65535,-2658,65535,-2518,65535,-2671,65535,-4277,65535,-3660,65535,-2982,65535,-1638,65535,-920,65535,-181,65535,-2558,65535,-2841,65535,-2086,65535,-3117,65535,-4394,65535,-5308,65535,-4627,65535,-3580,65535,-3864,65535,-3660,65535,-3020,65535,-3730,65535,-3443,65535,-3663,65535,-2966,65535,-2935,65535,-2851,65535,-1430,65535,-3411,65535,-5949,65535,-4123,65535,-4917,65535,-3175,65535,-4740,65535,-3368,65535,-3887,65535,-2278,65535,-2057,65535,-5533,65535,-3910,65535,-4989,65535,-5165,65535,-7344,65535,-8060,65535,-9212,65535,-8979,65535,-8184,65535,-10345,65535,-9363,65535,-8124,65535,-9710,65535,-7782,65535,-6446,65535,-7652,65535,-8124,65535,-9541,65535,-7441,65535,-7556,65535,-7509,65535,-6770,65535,-4213,65535,-5639,65535,-4888,65535,-4350,65535,-5881,65535,-5099,65535,-6892,65535,-4962,65535,-4096,65535,-6427,65535,-8012,65535,-7713,65535,-6274,65535,-6974,65535,-6914,65535,-6446,65535,-6818,65535,-5894,65535,-4472,65535,-4245,65535,-3643,65535,-3774,65535,-2295,65535,-3809,65535,-2054,65535,-3118,65535,-4158,65535,-6946,65535,-6594,65535,-6670,65535,-6170,65535,-7416,65535,-7109,65535,-7508,65535,-6062,65535,-7732,65535,-5163,65535,-7716,65535,-6322,65535,-8208,65535,-8236,65535,-8636,65535,-7540,65535,-6285,65535,-5926,65535,-4674,65535,-4715,65535,-5033,65535,-7153,65535,-9377,65535,-8956,65535,-7173,65535,-7842,65535,-7252,65535,-6658,65535,-7176,65535,-9280,65535,-8184,65535,-9093,65535,-9153,65535,-9653,65535,-9413,65535,-8840,65535,-7841,65535,-7109,65535,-7603,65535,-6370,65535,-7476,65535,-5801,65535,-3582,65535,-3438,65535,-5429,65535,-7272,65535,-7752,65535,-6386,65535,-6333,65535,-7457,65535,-7122,65535,-8272,65535,-8496,65535,-7864,65535,-5635,65535,-5146,65535,-7636,65535,-6782,65535,-6162,65535,-7155,65535,-8080,65535,-9589,65535,-9109,65535,-10581,65535,-10093,65535,-9965,65535,-9221,65535,-7731,65535,-5017,65535,-6720,65535,-5094,65535,-6462,65535,-6429,65535,-8272,65535,-8504,65535,-6797,65535,-4717,65535,-4842,65535,-5514,65535,-4330,65535,-6636,65535,-5785,65535,-4744,65535,-2491,65535,-1815,65535,-2955,65535,-3534,65535,-4045,65535,-7460,65535,-7160,65535,-7984,65535,-9153,65535,-9873,65535,-8681,65535,-8953,65535,-7108,65535,-9318,65535,-10577,65535,-10451,65535,-9633,65535,-8240,65535,-8521,65535,-8096,65535,-8435,65535,-8873,65535,-7952,65535,-6634,65535,-6033,65535,-6412,65535,-6661,65535,-6348,65535,-6314,65535,-6226,65535,-5522,65535,-6254,65535,-5964,65535,-6373,65535,-6709,65535,-5758,65535,-4174,65535,-7096,65535,-7683,65535,-8181,65535,-9449,65535,-8895,65535,-8853,65535,-6716,65535,-5606,65535,-5105,65535,-7842,65535,-6732,65535,-4554,65535,-5633,65535,-5329,65535,-7069,65535,-9501,65535,-7831,65535,-7860,65535,-6092,65535,-7107,65535,-9089,65535,-9498,65535,-7942,65535,-6684,65535,-7660,65535,-7152,65535,-8521,65535,-8325,65535,-9572,65535,-9644,65535,-9959,65535,-9489,65535,-8853,65535,-6884,65535,-5706,65535,-5996,65535,-6629,65535,-6325,65535,-7918,65535,-6348,65535,-6540,65535,-5446,65535,-6284,65535,-6156,65535,-7508,65535,-6119,65535,-7504,65535,-6476,65535,-5110,65535,-7961,65535,-7973,65535,-9685,65535,-9244,65535,-8322,65535,-9377,65535,-9089,65535,-9500,65535,-7726,65535,-5473,65535,-4595,65535,-4606,65535,-4426,65535,-7156,65535,-8276,65535,-7758,65535,-5645,65535,-7156,65535,-8265,65535,-9110,65535,-10211,65535,-8933,65535,-8822,65535,-8853,65535,-6836,65535,-7156,65535,-8276,65535,-7805,65535,-7138,65535,-9324,65535,-10127,65535,-12479,65535,-9971,65535,-11320,65535,-12619,65535,-12431,65535,-12950,65535,-13326,65535,-13478,65535,-10755,65535,-7945,65535,-9404,65535,-9336,65535,-9270,65535,-9179,65535,-10131,65535,-12079,65535,-13266,65535,-13334,65535,-13410,65535,-12858,65535,-10179,65535,-10924,65535,-9060,65535,-11252,65535,-10364,65535,-10207,65535,-9468,65535,-7753,65535,-5900,65535,-6472,65535,-5465,65535,-4933,65535,-5956,65535,-717565535,-4277,65535,-3790,65535,-3631,65535,-3119,65535,-4242,65535,-3591,65535,-3055,65535,-3985,65535,-4600,65535,-5770,65535,-3421,65535,-4873,65535,-3275,65535,-3469,65535,-3850,65535,-3909,65535,-4965,65535,-4005,65535,-5861,65535,-4063,65535,-6530,65535,-6297,65535,-5866,65535,-3913,65535,-4645,65535,-4451,65535,-5434,65535,-7045,65535,-7137,65535,-7864,65535,-5539,65535,-4674,65535,-4473,65535,-4473,65535,-4515,65535,-3906,65535,-2612,65535,-2655,65535,-4930,65535,-3818,65535,-3774,65535,-2359,65535,-4096,65535,-6482,65535,-4793,65535,-4148,65535,-5101,65535,-7220,65535,-6413,65535,-6666,65535,-6094,65535,-7864,65535,-5609,65535,-4420,65535,-4989,65535,-5181,65535,-6338,65535,-7334,65535,-8156,65535,-9062,65535,-11091,65535,-10669,65535,-8889,65535,-6954,65535,-5482,65535,-4101,65535,-6386,65535,-6210,65535,-6605,65535,-5722,65535,-4596,65535,-5258,65535,-4852,65535,-4665,65535,-5514,65535,-4292,65535,-4332,65535,-6514,65535,-4989,65535,-5283,65535,-5162,65535,-6476,65535,-4921,65535,-4777,65535,-2875,65535,-1661,65535,-3722,65535,-3264,65535,-3071,65535,-5514,65535,-4217,65535,-5130,65535,-6220,65535,-5466,65535,-5354,65535,-5379,65535,-5689,65535,-6334,65535,-7705,65535,-7016,65535,-8504,65535,-6674,65535,-5970,65535,-6162,65535,-7160,65535,-7924,65535,-6157,65535,-7416,65535,-7187,65535,-8256,65535,-7096,65535,-7651,65535,-7984,65535,-9077,65535,-10377,65535,-9749,65535,-7924,65535,-6072,65535,-6650,65535,-7428,65535,-8953,65535,-7048,65535,-8672,65535,-7332,65535,-8463,65535,-7661,65535,-6979,65535,-8320,65535,-9857,65535,-7224,65535,-6207,65535,-6839,65535,-6226,65535,-5514,65535,-4275,65535,-5970,65535,-6258,65535,-5686,65535,-6860,65535,-4862,65535,-3405,65535,-5840,65535,-5562,65535,-5790,65535,-4121,65535,-6693,65535,-6354,65535,-6338,65535,-7313,65535,-7089,65535,-8000,65535,-7764,65535,-5347,65535,-5782,65535,-5126,65535,-6701,65535,-6581,65535,-7140,65535,-9569,65535,-10192,65535,-10199,65535,-9931,65535,-9341,65535,-8933,65535,-8713,65535,-7280,65535,-7156,65535,-8133,65535,-10583,65535,-10195,65535,-10195,65535,-10448,65535,-9719,65535,-6805,65535,-6438,65535,-7460,65535,-7076,65535,-9105,65535,-10943,65535,-8993,65535,-8834,65535,-8372,65535,-8009,65535,-9593,65535,-8469,65535,-6837,65535,-6421,65535,-8533,65535,-7252,65535,-6536,65535,-5857,65535,-4478,65535,-3545,65535,-3517,65535,-3037,65535,-4442,65535,-5869,65535,-6252,65535,-5905,65535,-6092,65535,-7280,65535,-7092,65535,-7945,65535,-9404,65535,-9009,65535,-10371,65535,-10188,65535,-10279,65535,-9145,65535,-10207,65535,-9276,65535,-8421,65535,-10316,65535,-11055,65535,-13035,65535,-12431,65535,-12955,65535,-10707,65535,-8989,65535,-9445,65535,-9036,65535,-10755,65535,-7950,65535,-6460,65535,-6504,65535,-5825,65535,-6632,65535,-6044,65535,-8310,65535,-10207,65535,-9249,65535,-8421,65535,-9880,65535,-8089,65535,-8944,65535,-7340,65535,-7470,65535,-6204,65535,-7108,65535,-9364,65535,-9324,65535,-10308,65535,-11716,65535,-10115,65535,-10659,65535,-9452,65535,-8929,65535,-9007,65535,-10163,65535,-12203,65535,-12239,65535,-11967,65535,-11252,65535,-10462,65535,-8501,65535,-7068,65535,-9656,65535,-9060,65535,-11120,65535,-11683,65535,-11887,65535,-9724,65535,-7086,65535,-8325,65535,-9468,65535,-7781,65535,-6402,65535,-7577,65535,-8401,65535,-9068,65535,-11032,65535,-10920,65535,-9414,65535,-10661,65535,-10143,65535,-11316,65535,-12746,65535,-11032,65535,-11090,65535,-11875,65535,-10860,65535,-8784,65535,-6834,65535,-6882,65535,-5813,65535,-4989,65535,-6572,65535,-8081,65535,-9427,65535,-10140,65535,-12330,65535,-12842,65535,-11523,65535,-11171,65535,-13345,65535,-12314,65535,-13661,65535,-12548,65535,-12570,65535,-13058,65535,-15178,65535,-15534,65535,-12890,65535,-10536,65535,-9507,65535,-9783,65535,-7737,65535,-7257,65535,-8747,65535,-6953,65535,-7769,65535,-7445,65535,-8191,65535,-9031,65535,-10963,65535,-10928,65535,-10748,65535,-8636,65535,-9992,65535,-10963,65535,-11078,65535,-11778,65535,-9968,65535,-11998,65535,-13487,65535,-11762,65535,-11074,65535,-12158,65535,-14485,65535,-12611,65535,-11982,65535,-12222,65535,-12867,65535,-11206,65535,-10582,65535,-10490,65535,-9305,65535,-11746,65535,-9684,65535,-10032,65535,-12183,65535,-12835,65535,-11230,65535,-11218,65535,-12215,65535,-12867,65535,-11374,65535,-13451,65535,-12067,65535,-13227,65535,-12775,65535,-10614,65535,-10486,65535,-9887,65535,-10691,65535,-10057,65535,-12432,65535,-12562,65535,-12943,65535,-13071,65535,-13809,65535,-13304,65535,-14640,65535,-13733,65535,-11600,65535,-11991,65535,-13989,65535,-15393,65535,-16437,65535,-14485,65535,-12688,65535,-11762,65535,-11298,65535,-13681,65535,-12578,65535,-11811,65535,-11927,65535,-11134,65535,-13055,65535,-15053,65535,-16425,65535,-15473,65535,-13701,65535,-11394,65535,-11879,65535,-12375,65535,-13937,65535,-14197,65535,-13227,65535,-12846,65535,-12830,65535,-11566,65535,-12562,65535,-13067,65535,-14352,65535,-14117,65535,-16094,65535,-16286,65535,-17544,65535,-17061,65535,-16857,65535,-14485,65535,-12750,65535,-12343,65535,-13813,65535,-13559,65535,-13973,65535,-14005,65535,-14197,65535,-13231,65535,-12615,65535,-11818,65535,-11454,65535,-10650,65535,-10534,65535,-11394,65535,-11959,65535,-13691,65535,-12759,65535,-11943,65535,-12311,65535,-13583,65535,-12499,65535,-13047,65535,-15229,65535,-14177,65535,-14673,65535,-14085,65535,-16038,65535,-17544,65535,-17045,65535,-18172,65535,-17567,65535,-15674,65535,-14197,65535,-13304,65535,-14768,65535,-12296,65535,-14592,65535,-14809,65535,-14213,65535,-14560,65535,-14700,65535,-12311,65535,-13673,65535,-13231,65535,-12658,65535,-13609,65535,-12766,65535,-11090,65535,-13817,65535,-13295,65535,-15345,65535,-14912,65535,-14684,65535,-13681,65535,-12818,65535,-12606,65535,-12402,65535,-14297,65535,-15674,65535,-14352,65535,-14224,65535,-13577,65535,-12683,65535,-11975,65535,-12610,65535,-12067,65535,-13481,65535,-12103,65535,-13247,65535,-14046,65535,-16142,65535,-17839,65535,-16457,65535,-15613,65535,-16062,65535,-16090,65535,-16602,65535,-15245,65535,-15449,65535,-13631,65535,-14007,65535,-14405,65535,-15465,65535,-14885,65535,-14043,65535,-16309,65535,-16714,65535,-14839,65535,-12650,65535,-12154,65535,-14715,65535,-11567,65535,-12118,65535,-13085,65535,-13651,65535,-12494,65535,-10824,65535,-10576,65535,-11107,65535,-13169,65535,-15070,65535,-16101,65535,-17004,65535,-17298,65535,-17226,65535,-15214,65535,-13377,65535,-12846,65535,-10652,65535,-11251,65535,-10520,65535,-10912,65535,-9347,65535,-11316,65535,-13137,65535,-14711,65535,-11830,65535,-11087,65535,-11978,65535,-13217,65535,-11922,65535,-11823,65535,-11183,65535,-12650,65535,-11874,65535,-10151,65535,-11471,65535,-10088,65535,-11219,65535,-10252,65535,-9607,65535,-10284,65535,-9824,65535,-9639,65535,-10396,65535,-11862,65535,-11219,65535,-10188,65535,-11555,65535,-11427,65535,-12706,65535,-11319,65535,-13066,65535,-14703,65535,-12730,65535,-9719,65535,-7338,65535,-8113,65535,-9816,65535,-10152,65535,-11583,65535,-11779,65535,-10556,65535,-10632,65535,-10062,65535,-12079,65535,-13570,65535,-14007,65535,-14831,65535,-13322,65535,-13873,65535,-12582,65535,-12602,65535,-11124,65535,-12031,65535,-11907,65535,-11240,65535,-11459,65535,-10839,65535,-10239,65535,-9911,65535,-8357,65535,-9872,65535,-8524,65535,-7953,65535,-8848,65535,-7246,65535,-7102,65535,-9779,65535,-8054,65535,-11200,65535,-11580,65535,-11711,65535,-10403,65535,-10335,65535,-10236,65535,-9635,65535,-10572,65535,-11587,65535,-11491,65535,-11016,65535,-12378,65535,-11731,65535,-8848,65535,-7210,65535,-7370,65535,-8133,65535,-10604,65535,-9430,65535,-9735,65535,-86738,65535,-11240,65535,-11335,65535,-11907,65535,-11000,65535,-10844,65535,-9951,65535,-10239,65535,-9640,65535,-10403,65535,-10271,65535,-9724,65535,-6964,65535,-7501,65535,-6072,65535,-8833,65535,-8585,65535,-8708,65535,-7443,65535,-7652,65535,-8329,65535,-9193,65535,-9585,65535,-9499,65535,-7620,65535,-8073,65535,-10173,65535,-11775,65535,-9921,65535,-7794,65535,-5157,65535,-6737,65535,-4099,65535,-5898,65535,-4341,65535,-6350,65535,-6510,65535,-5395,65535,-7220,65535,-6594,65535,-6802,65535,-4494,65535,-3495,65535,-2615,65535,-3559,65535,-2955,65535,-3407,65535,-3855,65535,-4590,65535,-3671,65535,-4901,65535,-4485,65535,-4155,655365535,-16233,65535,-16473,65535,-14512,65535,-13071,65535,-13849,65535,-13545,65535,-14681,65535,-13449,65535,-12048,65535,-14448,65535,-12706,65535,-10442,65535,-10821,65535,-10714,65535,-8844,65535,-7971,65535,-9124,65535,-12563,65535,-13091,65535,-14961,65535,-15837,65535,-13135,65535,-14179,65535,-14379,65535,-15137,65535,-15361,65535,-14507,65535,-13827,65535,-12734,65535,-11446,65535,-11196,65535,-11823,65535,-11462,65535,-9817,65535,-10412,65535,-12654,65535,-11650,65535,-11203,65535,-11638,65535,-12266,65535,-12686,65535,-12026,65535,-12246,65535,-11507,65535,-9964,65535,-10364,65535,-11687,65535,-11199,65535,-11938,65535,-10604,65535,-9655,65535,-9304,65535,-11539,65535,-10042,65535,-11718,65535,-9127,65535,-11731,65535,-9105,65535,-10816,65535,-11075,65535,-13021,65535,-13201,65535,-13009,65535,-14135,65535,-15633,65535,-15086,65535,-14759,65535,-11507,65535,-9831,65535,-9427,65535,-10284,65535,-9799,65535,-9239,65535,-8924,65535,-9447,65535,-9395,65535,-10010,65535,-11459,65535,-10908,65535,-10508,65535,-9063,65535,-11411,65535,-11379,65535,-11048,65535,-12538,65535,-10828,65535,-10968,65535,-10836,65535,-10364,65535,-11583,65535,-11907,65535,-11251,65535,-10376,65535,-10588,65535,-10780,65535,-9847,65535,-8005,65535,-10131,65535,-12346,65535,-11619,65535,-11731,65535,-8908,65535,-8036,65535,-9939,65535,-8980,65535,-9639,65535,-10271,65535,-9982,65535,-10143,65535,-11452,65535,-10996,65535,-11283,65535,-10387,65535,-11724,65535,-9144,65535,-10572,65535,-11631,65535,-10828,65535,-10844,65535,-10115,65535,-10907,65535,-10108,65535,-11072,65535,-12938,65535,-12143,65535,-13709,65535,-12079,65535,-13453,65535,-12527,65535,-11843,65535,-10892,65535,-8843,65535,-7406,65535,-7989,65535,-8591,65535,-8341,65535,-8421,65535,-10143,65535,-11336,65535,-11955,65535,-10019,65535,-10319,65535,-11512,65535,-9500,65535,-8062,65535,-10575,65535,-10828,65535,-10836,65535,-10319,65535,-11463,65535,-10531,65535,-8940,65535,-10403,65535,-10207,65535,-9544,65535,-9955,65535,-9820,65535,-9927,65535,-9772,65535,-8661,65535,-7930,65535,-8329,65535,-9324,65535,-10240,65535,-9449,65535,-8961,65535,-8853,65535,-6964,65535,-7324,65535,-8725,65535,-6348,65535,-6476,65535,-5178,65535,-8072,65535,-9997,65535,-9548,65535,-9461,65535,-8629,65535,-7624,65535,-7952,65535,-6754,65535,-5635,65535,-5238,65535,-6610,65535,-5593,65535,-5861,65535,-4222,65535,-5373,65535,-5304,65535,-5093,65535,-7397,65535,-6865,65535,-4805,65535,-3155,65535,-6946,65535,-6642,65535,-5613,65535,-5781,65535,-3593,65535,-3439,65535,-3856,65535,-4609,65535,-5416,65535,-6830,65535,-5693,65535,-4933,65535,-4526,65535,-3363,65535,-4814,65535,-2720,65535,-2791,65535,-1091,65535,-2188,65535,-3282,65535,-2807,65535,-2314,65535,-2038,65535,-4500,65535,-2668,65535,-1807,65535,-1555,65535,-1910,65535,-1215,65535,-3170,65535,-5409,65535,-4855,65535,-5194,65535,-7481,65535,-6057,65535,-5917,65535,-3758,65535,-1427,65535,-3107,65535,-5322,65535,-6009,65535,-6910,65535,-4581,65535,-4436,65535,-4521,65535,-4377,65535,-4085,65535,-4415,65535,-6553,65535,-5786,65535,-3075,65535,-3293,65535,-4538,65535,-2567,65535,-1782,65535,-1156,65535,-3246,65535,-2556,65535,-2477,65535,-2614,65535,-3299,65535,-4135,65535,-6308,65535,-5281,65535,-4219,65535,-5039,65535,-4443,65535,-4314,65535,-3354,65535,-4738,65535,-3679,65535,-4299,65535,-4739,65535,-3468,65535,-3827,65535,-1464,65535,-3196,65535,-4133,65535,-4811,65535,-3532,65535,-2227,65535,-3679,65535,-4522,65535,-3868,65535,-3330,65535,-3967,65535,-3792,65535,-1244,65535,-2286,65535,-1737,65535,-1493,65535,-1260,65535,-1046,65535,-1462,65535,-3219,65535,-4347,65535,-5793,65535,-2751,65535,-1381,65535,-1253,65535,-1814,65535,-1236,65535,-2253,65535,-1464,65535,-3123,65535,-4699,65535,-3564,65535,-2336,65535,-2686,65535,-3203,65535,-3951,65535,-2388,65535,-3952,65535,-2091,65535,-3628,65535,-2586,65535,-2601,65535,-1778,65535,-1174,65535,-2140,65535,-2779,65535,-1680,65535,-2950,65535,-1595,65535,-1122,65535,-3168,65535,-4099,65535,-5905,65535,-4778,65535,-3168,65535,-4037,65535,-4163,65535,-6345,65535,-7366,65535,-6405,65535,-5953,65535,-3580,65535,-3651,65535,-3922,65535,-1621,65535,-1935,65535,-1994,65535,-1282,65535,-1893,65535,-142,65535,-1394,65535,-3180,65535,-4243,65535,-3372,65535,-3263,65535,-2366,65535,-3532,65535,-2107,65535,-2382,65535,-2266,65535,-2862,65535,-3171,65535,-3570,65535,-1873,65535,-1542,65535,-518,65535,-1214,65535,-3982,65535,-2353,65535,-4477,65535,-4459,65535,-5521,65535,-4723,65535,-3980,65535,-2740,65535,-2108,65535,-4147,65535,-5162,65535,-7290,65535,-7194,65535,-8956,65535,-7812,65535,-4506,65535,-2819,65535,-1562,65535,-1087,65535,-2563,65535,-2120,65535,-3279,65535,-2882,65535,-1630,65535,-1205,65535,-3925,65535,-4181,65535,-5201,65535,-7237,65535,-5944,65535,-4270,65535,-3946,65535,-2822,65535,-731,65535,-1222,65535,-3199,65535,-5997,65535,-6982,65535,-6854,65535,-6014,65535,-5949,65535,-4051,65535,-4773,65535,-3614,65535,-4019,65535,-4638,65535,-5277,65535,-4526,65535,-3434,65535,-4006,65535,-5820,65535,-3279,65535,-2931,65535,-3495,65535,-2501,65535,-1536,65535,-522,65535,-1440,65535,-2036,65535,-4862,65535,-4516,65535,-4245,65535,-3547,65535,-3759,65535,-1562,65535,-1221,65535,-2919,65535,-1666,65535,-1456,65535,-3315,65535,-3135,65535,-5532,65535,-3855,65535,-4471,65535,-5344,65535,-7189,65535,-9123,65535,-10105,65535,-9597,65535,-9347,65535,-9379,65535,-9646,65535,-7841,65535,-6892,65535,-5653,65535,-4948,65535,-3246,65535,-2704,65535,-4165,65535,-6685,65535,-5256,65535,-5982,65535,-5930,65535,-4144,65535,-5197,65535,-7649,65535,-6422,65535,-6377,65535,-7697,65535,-6782,65535,-4043,65535,-5929,65535,-3863,65535,-3791,65535,-1806,65535,-1984,65535,-963,65535,-1978,65535,-1517,65535,-2187,65535,-3914,65535,-4164,65535,-6222,65535,-6506,65535,-6334,65535,-7995,65535,-8620,65535,-6834,65535,-6734,65535,-5117,65535,-5946,65535,-5581,65535,-5533,65535,-3947,65535,-2994,65535,-3223,65535,-4342,65535,-6414,65535,-7412,65535,-8039,65535,-7996,65535,-8840,65535,-7736,65535,-4750,65535,-2522,65535,-2374,65535,-2599,65535,-1926,65535,-3323,65535,-4741,65535,-3422,65535,-4948,65535,-2896,65535,-2887,65535,-1352,65535,-3899,65535,-3930,65535,-3762,65535,-982,65535,-622,65535,-1215,65535,-3116,65535,-4091,65535,-4629,65535,-3507,65535,-1998,65535,-2494,65535,-2459,65535,-4309,65535,-4187,65535,-5131,65535,-7317,65535,-7525,65535,-5353,65535,-6654,65535,-5414,65535,-6942,65535,-4917,65535,-3158,65535,-6336,65535,-8232,65535,-7040,65535,-7029,65535,-8312,65535,-8720,65535,-6834,65535,-6834,65535,-6722,65535,-5302,65535,-5005,65535,-6686,65535,-7429,65535,-7038,65535,-7384,65535,-6861,65535,-5020,65535,-5205,65535,-6572,65535,-5213,65535,-6477,65535,-4873,65535,-3247,65535,-3023,65535,-3762,65535,-3114,65535,-4612,65535,-3969,65535,-2854,65535,-3143,65535,-4693,65535,-5866,65535,-4015,65535,-4921,65535,-4869,65535,-3581,65535,-3059,65535,-6366,65535,-7833,65535,-7652,65535,-8028,65535,-9091,65535,-10253,65535,-8584,65535,-8480,65535,-6331,65535,-7586,65535,-8012,65535,-7736,65535,-4857,65535,-4382,65535,-6126,65535,-8096,65535,-8251,65535,-8224,65535,-6842,65535,-6270,65535,-5466,65535,-5277,65535,-4728,65535,-3606,65535,-4222,65535,-5205,65535,-5958,65535,-4921,65535,-4867,65535,-3709,65535,-3941,65535,-5161,65535,-7896,65535,-5738,65535,-3303,65535,-3578,65535,-3261,65535,-4149,65535,-5315,65535,-5402,65535,-6898,65535,-4949,65535,-5466,65535,-5274,65535,-6272,65535,-6735,65535,-4035,65535,-6206,65535,-6946,65535,-6654,65535,-7400,65535,-8544,65535,-6690,65535,-7217,65535,-6530,65535,-6426,65535,-8096,65535,-8357,65535,-7092,65535,-7968,65535,-7672,65535,-6722,65535,-5353,65535,-5162,65535,-6430,65535,-8224,65535,-6968,65535,-6952,65535,-5770,65535,-3600,65535,-4265,65535,-4076,65535,-7204,65535,-7902,65535,-7331,65535,-8176,65535,-10248,65535,-8777,65535,-7473,65535,-5845,65535,-5285,65535,-5142,65535,-7876,65535,-7186,65535,-8495,65535,-7736,65535,-4986,65535,-6829,65535,-7096,65535,-7652,65535,-8032,65535,-8064,65535,-10503,65535,-8144,65535,-9916,65535,-7996,65535,-8155,65535,-8965,65535,-8617,65535,-8601,65535,-7272,65535,-7918,65535,-6146,65535,-8505,65535,-6709,65535,-5798,65535,-3910,65535,-4766,65535,-2832,65535,-1804,65535,-870,]
data_array = np.array(data)

data2 = [65535,-16321,65535,-16449,65535,-16533,65535,-16652,65535,-16742,65535,-16806,65535,-16920,65535,-17084,65535,-17140,65535,-17252,65535,-17396,65535,-17492,65535,-17556,65535,-17643,65535,-17771,65535,-17915,65535,-18007,65535,-18071,65535,-18131,65535,-18259,65535,-18403,65535,-18507,65535,-18628,65535,-18678,65535,-18810,65535,-18886,65535,-19011,65535,-19125,65535,-19202,65535,-19282,65535,-19410,65535,-19442,65535,-19602,65535,-19763,65535,-19847,65535,-19911,65535,-20019,65535,-20107,65535,-20226,65535,-20270,65535,-20370,65535,-20450,65535,-20594,65535,-20762,65535,-20794,65535,-20905,65535,-20954,65535,-21130,65535,-21213,65535,-21325,65535,-21409,65535,-21533,65535,-21593,65535,-21721,65535,-21781,65535,-21865,65535,-21929,65535,-22073,65535,-22172,65535,-22281,65535,-22368,65535,-22448,65535,-22560,65535,-22632,65535,-22728,65535,-22852,65535,-22936,65535,-23041,65535,-23108,65535,-23231,65535,-23355,65535,-23375,65535,-23544,65535,-23567,65535,-23679,65535,-23783,65535,-23823,65535,-23963,65535,-24041,65535,-24154,65535,-24230,65535,-24294,65535,-24390,65535,-24543,65535,-24594,65535,-24666,65535,-24790,65535,-24910,65535,-24942,65535,-25046,65535,-25118,65535,-25217,65535,-25286,65535,-25381,65535,-25429,65535,-25550,65535,-25565,65535,-25709,65535,-25817,65535,-25885,65535,-25949,65535,-26061,65535,-26141,65535,-26224,65535,-26308,65535,-26388,65535,-26436,65535,-26564,65535,-26644,65535,-26728,65535,-26808,65535,-26924,65535,-26956,65535,-27060,65535,-27128,65535,-27164,65535,-27247,65535,-27315,65535,-27379,65535,-27556,65535,-27587,65535,-27603,65535,-27805,65535,-27820,65535,-27868,65535,-27941,65535,-27980,65535,-28092,65535,-28156,65535,-28196,65535,-28316,65535,-28317,65535,-28444,65535,-28564,65535,-28592,65535,-28628,65535,-28715,65535,-28812,65535,-28820,65535,-28887,65535,-28987,65535,-29068,65535,-29083,65535,-29147,65535,-29203,65535,-29251,65535,-29327,65535,-29411,65535,-29475,65535,-29571,65535,-29603,65535,-29658,65535,-29738,65535,-29819,65535,-29834,65535,-29898,65535,-29955,65535,-30075,65535,-30086,65535,-30122,65535,-30150,65535,-30181,65535,-30226,65535,-30330,65535,-30402,65535,-30466,65535,-30462,65535,-30578,65535,-30586,65535,-30610,65535,-30713,65535,-30713,65535,-30834,65535,-30834,65535,-30841,65535,-30841,65535,-30962,65535,-31082,65535,-31082,65535,-31097,65535,-31097,65535,-31161,65535,-31217,65535,-31249,65535,-31329,65535,-31339,65535,-31345,65535,-31401,65535,-31409,65535,-31409,65535,-31505,65535,-31593,65535,-31593,65535,-31633,65535,-31658,65535,-31667,65535,-31665,65535,-31731,65535,-31835,65535,-31835,65535,-31903,65535,-31963,65535,-31907,65535,-31955,65535,-31963,65535,-31971,65535,-31964,65535,-32091,65535,-32099,65535,-32095,65535,-32115,65535,-32115,65535,-32163,65535,-32163,65535,-32218,65535,-32234,65535,-32218,65535,-32331,65535,-32278,65535,-32340,65535,-32346,65535,-32339,65535,-32362,65535,-32406,65535,-32410,65535,-32406,65535,-32410,65535,-32474,65535,-32470,65535,-32474,65535,-32490,65535,-32474,65535,-32474,65535,-32595,65535,-32595,65535,-32595,65535,-32595,65535,-32595,65535,-32596,65535,-32595,65535,-32634,65535,-32614,65535,-32598,65535,-32614,65535,-32618,65535,-32602,65535,-32595,65535,-32602,65535,-32618,65535,-32618,65535,-32618,65535,-32660,65535,-32634,65535,-32634,65535,-32602,65535,-32602,65535,-32595,65535,-32618,65535,-32602,65535,-32596,65535,-32598,65535,-32595,65535,-32587,65535,-32595,65535,-32595,65535,-32587,65535,-32490,65535,-32587,65535,-32470,65535,-32474,65535,-32474,65535,-32406,65535,-32410,65535,-32410,65535,-32362,65535,-32346,65535,-32362,65535,-32404,65535,-32342,65535,-32339,65535,-32339,65535,-32331,65535,-32206,65535,-32218,65535,-32218,65535,-32175,65535,-32163,65535,-32111,65535,-32099,65535,-32091,65535,-32091,65535,-32083,65535,-31987,65535,-32083,65535,-31967,65535,-31963,65535,-31859,65535,-31843,65535,-31859,65535,-31827,65535,-31835,65535,-31835,65535,-31715,65535,-31722,65535,-31617,65535,-31601,65535,-31585,65535,-31585,65535,-31489,65535,-31466,65535,-31469,65535,-31361,65535,-31345,65535,-31337,65535,-31337,65535,-31249,65535,-31188,65535,-31161,65535,-31097,65535,-31093,65535,-31082,65535,-30985,65535,-30962,65535,-30901,65535,-30841,65535,-30834,65535,-30834,65535,-30722,65535,-30654,65535,-30654,65535,-30586,65535,-30578,65535,-30462,65535,-30460,65535,-30354,65535,-30322,65535,-30322,65535,-30170,65535,-30153,65535,-30090,65535,-30083,65535,-30075,65535,-29958,65535,-29898,65535,-29830,65535,-29754,65535,-29674,65535,-29619,65535,-29603,65535,-29571,65535,-29475,65535,-29411,65535,-29323,65535,-29323,65535,-29219,65535,-29147,65535,-29083,65535,-29076,65535,-28971,65535,-28948,65535,-28843,65535,-28812,65535,-28692,65535,-28580,65535,-28608,65535,-28572,65535,-28446,65535,-28340,65535,-28316,65535,-28228,65535,-28172,65535,-28076,65535,-28061,65535,-27948,65535,-27880,65535,-27813,65535,-27724,65535,-27587,65535,-27556,65535,-27556,65535,-27395,65535,-27315,65535,-27203,65535,-27164,65535,-27128,65535,-27052,65535,-27052,65535,-26824,65535,-26808,65535,-26716,65535,-26624,65535,-26557,65535,-26448,65535,-26388,65535,-26304,65535,-26196,65535,-26077,65535,-26061,65535,-26045,65535,-25881,65535,-25869,65535,-25789,65535,-25669,65535,-25553,65535,-25461,65535,-25381,65535,-25294,65535,-25205,65535,-25134,65535,-25046,65535,-24942,65535,-24910,65535,-24790,65535,-24666,65535,-24582,65535,-24544,65535,-24418,65535,-24353,65535,-24198,65535,-24154,65535,-24059,65535,-23931,65535,-23911,65535,-23784,65535,-23663,65535,-23567,65535,-23471,65535,-23435,65535,-23307,65535,-23288,65535,-23128,65535,-23040,65535,-22936,65535,-22856,65535,-22792,65535,-22660,65535,-22576,65535,-22464,65535,-22384,65535,-22281,65535,-22192,65535,-22089,65535,-22034,65535,-21909,65535,-21785,65535,-21673,65535,-21537,65535,-21530,65535,-21389,65535,-21277,65535,-21185,65535,-21114,65535,-21028,65535,-20922,65535,-20794,65535,-20714,65535,-20574,65535,-20530,65535,-20418,65535,-20290,65535,-20267,65535,-20146,65535,-20043,65535,-19911,65535,-19847,65535,-19707,65535,-19586,65535,-19515,65535,-19394,65535,-19330,65535,-19214,65535,-19078,65535,-19034,65535,-18890,65535,-18778,65535,-18682,65535,-18559,65535,-18563,65535,-18381,65535,-18275,65535,-18175,65535,-18027,65535,-18004,65535,-17876,65535,-17803,65535,-17703,65535,-17556,65535,-17504,65535,-17332,65535,-17245,65535,-17184,65535,-17048,65535,-17020,65535,-16872,65535,-16748,65535,-16636,65535,-16497,65535,-16485,65535,-16325,65535,-16197,65535,-16133,65535,-16061,65535,-15982,65535,-15869,65535,-15734,65535,-15628,65535,-15486,65535,-15478,65535,-15294,65535,-15166,65535,-15118,65535,-14998,65535,-14929,65535,-14790,65535,-14719,65535,-14642,65535,-14475,65535,-14426,65535,-14255,65535,-14215,65535,-14139,65535,-13968,65535,-13889,65535,-13795,65535,-13712,65535,-13584,65535,-13472,65535,-13456,65535,-13264,65535,-13216,65535,-13087,65535,-12961,65535,-12872,65535,-12825,65535,-12728,65535,-12588,65535,-12449,65535,-12401,65535,-12289,65535,-12145,65535,-12080,65535,-11977,65535,-11849,65535,-11785,65535,-11690,65535,-11624,65535,-11408,65535,-11408,65535,-11305,65535,-11177,65535,-11088,65535,-11045,65535,-10905,65535,-10817,65535,-10690,65535,-10617,65535,-10493,65535,-10442,65535,-10330,65535,-10209,65535,-10145,65535,-10021,65535,-9946,65535,-9830,65535,-9810,65535,-9698,65535,-9530,65535,-9457,65535,-9362,65535,-9282,65535,-9230,65535,-9074,65535,-9035,65535,-8931,65535,-8807,65535,-8699,65535,-8647,65535,-8603,65535,-8451,65535,-8420,65535,-8303,65535,-8146,65535,-8083,65535,-8047,65535,-7972,65535,-7848,65535,-7724,65535,-7707,65535,-7580,65535,-7484,65535,-7429,65535,-7340,65535,-7260,65535,-7190,65535,-7093,65535,-7013,65535,-6885,65535,-6881,65535,-6767,65535,-6639,65535,-6597,65535,-6529,65535,-6445,65535,-6333,65535,-6252,65535,-6152,65535,-6045,65535,-6033,65535,-5958,65535,-5838,65535,-5774,65535,-5682,65535,-5655,65535,-5534,65535,-5454,65535,-5434,65535,-5306,65535,-5209,65535,-5165,65535,-5070,65535,-5014,65535,-4906,65535,-4879,65535,-4807,65535,-4723,65535,-4655,65535,-4565,65535,-4523,65535,-4431,65535,-4367,65535,-4303,65535,-4216,65535,-4143,65535,-4095,65535,-4044,65535,-3972,65535,-3848,65535,-3832,65535,-3761,65535,-3648,65535,-3618,65535,-3574,65535,-3481,65535,-3422,65535,-3327,65535,-3319,65535,-3211,65535,-3151,65535,-3128,65535,-3071,65535,-2983,65535,-2952,65535,-2880,65535,-2859,65535,-2752,65535,-2695,65535,-2599,65535,-2583,65535,-2535,65535,-2455,65535,-2468,65535,-2320,65535,-2320,65535,-2218,65535,-2237,65535,-2171,65535,-2064,65535,-2042,65535,-1972,65535,-1987,65535,-1894,65535,-1807,65535,-1824,65535,-1775,65535,-1719,65535,-1640,65535,-1619,65535,-1576,65535,-1495,65535,-1527,65535,-1453,65535,-1352,65535,-1331,65535,-1291,65535,-1280,65535,-1217,65535,-1162,65535,-1159,65535,-1106,65535,-1037,65535,-1009,65535,-1011,65535,-934,65535,-947,65535,-928,65535,-854,65535,-785,65535,-793,65535,-802,65535,-697,65535,-697,65535,-678,65535,-649,65535,-625,65535,-566,65535,-521,65535,-473,65535,-423,65535,-462,65535,-354,65535,-377,65535,-280,65535,-250,65535,-214,65535,-218,65535,-222,65535,-146,65535,-154,65535,-153,65535,-144,65535,-146,65535,-174,65535,-144,65535,-158,65535,-162,65535,-88,65535,-212,65535,-170,65535,-192,65535,-190,65535,-146,65535,-142,65535,-89,65535,-129,65535,-176,65535,-146,65535,-198,65535,-146,65535,-142,65535,-152,65535,-160,65535,-178,65535,-130,65535,-146,65535,-190,65535,-126,65535,-146,65535,-144,65535,-176,65535,-165,65535,-162,65535,-160,65535,-194,65535,-138,65535,-162,65535,-128,65535,-161,65535,-140,65535,-144,65535,-164,65535,-162,65535,-130,65535,-153,65535,-154,65535,-142,65535,-129,65535,-130,65535,-138,65535,-158,65535,-162,65535,-162,65535,-174,65535,-144,65535,-152,65535,-178,65535,-193,65535,-160,65535,-136,65535,-160,65535,-165,65535,-128,65535,-164,65535,-144,65535,-181,65535,-218,65535,-266,65535,-254,65535,-304,65535,-382,65535,-415,65535,-414,65535,-442,65535,-470,65535,-537,65535,-505,65535,-618,65535,-656,65535,-657,65535,-694,65535,-738,65535,-760,65535,-784,65535,-840,65535,-876,65535,-902,65535,-898,65535,-984,65535,-978,65535,-1009,65535,-1023,65535,-1101,65535,-1120,65535,-1192,65535,-1269,65535,-1295,65535,-1281,65535,-1337,65535,-1384,65535,-1439,65535,-1467,65535,-1528,65535,-1585,65535,-1592,65535,-1682,65535,-1720,65535,-1728,65535,-1755,65535,-1857,65535,-1941,65535,-1960,65535,-2019,65535,-2019,65535,-2064,65535,-2107,65535,-2201,65535,-2288,65535,-2283,65535,-2408,65535,-2400,65535,-2470,65535,-2484,65535,-2554,65535,-2656,65535,-2688,65535,-2775,65535,-2823,65535,-2839,65535,-2903,65535,-2978,65535,-3127,65535,-3099,65535,-3192,65535,-3246,65535,-3295,65535,-3327,65535,-3395,65535,-3447,65535,-3569,65535,-3632,65535,-3732,65535,-3768,65535,-3799,65535,-3857,65535,-3987,65535,-4035,65535,-4136,65535,-4144,65535,-4216,65535,-4300,65535,-4383,65535,-4447,65535,-4529,65535,-4583,65535,-4656,65535,-4693,65535,-4803,65535,-4839,65535,-4886,65535,-5037,65535,-5079,65535,-5159,65535,-5213,65535,-5309,65535,-5407,65535,-5463,65535,-5541,65535,-5590,65535,-5669,65535,-5790,65535,-5862,65535,-5918,65535,-5998,65535,-6093,65535,-6125,65535,-6234,65535,-6317,65535,-6406,65535,-6509,65535,-6591,65535,-6677,65535,-6782,65535,-6877,65535,-6899,65535,-7006,65535,-7079,65535,-7176,65535,-7221,65535,-7320,65535,-7413,65535,-7484,65535,-7564,65535,-7692,65535,-7756,65535,-7844,65535,-7916,65535,-8050,65535,-8126,65535,-8227,65535,-8307,65535,-8381,65535,-8461,65535,-8515,65535,-8677,65535,-8723,65535,-8858,65535,-8902,65535,-9003,65535,-9058,65535,-9202,65535,-9291,65535,-9387,65535,-9458,65535,-9562,65535,-9641,65535,-9734,65535,-9882,65535,-9955,65535,-10026,65535,-10129,65535,-10241,65535,-10337,65535,-10385,65535,-10529,65535,-10617,65535,-10697,65535,-10817,65535,-10856,65535,-10969,65535,-11084,65535,-11200,65535,-11308,65535,-11441,65535,-11472,65535,-11609,65535,-11698,65535,-11769,65535,-11800,65535,-11961,65535,-12096,65535,-12129,65535,-12289,65535,-12353,65535,-12457,65535,-12552,65535,-12664,65535,-12776,65535,-12856,65535,-12968,65535,-13032,65535,-13136,65535,-13274,65535,-13360,65535,-13484,65535,-13585,65535,-13671,65535,-13767,65535,-13847,65535,-13991,65535,-14067,65535,-14207,65535,-14271,65535,-1435406,65535,-15546,65535,-15613,65535,-15737,65535,-15862,65535,-15885,65535,-16077,65535,-16165,65535,-16197,65535,-16325,65535,-16485,65535,-16513,65535,-16636,65535,-16741,65535,-16869,65535,-16924,65535,-17080,65535,-17172,65535,-17252,65535,-17332,65535,-17412,65535,-17572,65535,-17675,65535,-17767,65535,-17883,65535,-17915,65535,-18091,65535,-18175,65535,-18259,65535,-18381,65535,-18451,65535,-18627,65535,-18682,65535,-18794,65535,-18906,65535,-19011,65535,-19138,65535,-19182,65535,-19266,65535,-19390,65535,-19506,65535,-19655,65535,-19707,65535,-19815,65535,-19915,65535,-20019,65535,-20091,65535,-20162,65535,-20350,65535,-2039665535,-4807,65535,-4855,65535,-4895,65535,-5034,65535,-5082,65535,-5159,65535,-5213,65535,-5278,65535,-5431,65535,-5431,65535,-5582,65535,-5590,65535,-5714,65535,-5762,65535,-5878,65535,-5966,65535,-5999,65535,-6109,65535,-6166,65535,-6246,65535,-6329,65535,-6391,65535,-6509,65535,-6565,65535,-6671,65535,-6725,65535,-6821,65535,-6864,65535,-7024,65535,-7125,65535,-7192,65535,-7224,65535,-7308,65535,-7404,65535,-7484,65535,-7596,65535,-7652,65535,-7755,65535,-7852,65535,-7917,65535,-8011,65535,-8099,65535,-8194,65535,-8291,65535,-8387,65535,-8495,65535,-8534,65535,-8667,65535,-8739,65535,-8819,65535,-8932,65535,-9019,65535,-9106,65535,-9197,65535,-9307,65535,-9378,65535,-9458,65535,-9550,65535,-9689,65535,-9786,65535,-9858,65535,-9939,65535,-10026,65535,-10106,65535,-10225,65535,-10365,65535,-10465,65535,-10465,65535,-10633,65535,-10696,65535,-10817,65535,-10873,65535,-11001,65535,-11088,65535,-11185,65535,-11276,65535,-11372,65535,-11472,65535,-11570,65535,-11715,65535,-11826,65535,-11875,65535,-11977,65535,-12041,65535,-12145,65535,-12329,65535,-12353,65535,-12477,65535,-12600,65535,-12648,65535,-12760,65535,-12904,65535,-12953,65535,-13064,65535,-13136,65535,-13296,65535,-13360,65535,-13468,65535,-13585,65535,-13671,65535,-13783,65535,-13847,65535,-13991,65535,-14107,65535,-14175,65535,-14299,65535,-14395,65535,-14495,65535,-14610,65535,-14726,65535,-14839,65535,-14870,65535,-14998,65535,-15114,65535,-15182,65535,-15274,65535,-15406,65535,-15545,65535,-15629,65535,-15737,65535,-15854,65535,-15884,65535,-16029,65535,-16117,65535,-16239,]
data_array2 = np.array(data2)

len(data)

In [ ]:
import matplotlib.pyplot as plt

#plt.plot(data_array[1::2])
#plt.plot(data_array[::2])
plt.plot(data_array2[1::2])
plt.plot(data_array2[::2])

In [ ]:
%pylab
plt.plot(data_array2[1::2])
plt.plot(data_array2[::2])